In [1]:
import json, os, sys, random, time, copy
from collections import Counter, defaultdict
import numpy as np
from bs4 import BeautifulSoup
import urllib.parse
import urllib.request
from pprint import pprint

In [2]:
USER_AGENT_LIST = [
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; AcooBrowser; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.0; Acoo Browser; SLCC1; .NET CLR 2.0.50727; Media Center PC 5.0; .NET CLR 3.0.04506)",
            "Mozilla/4.0 (compatible; MSIE 7.0; AOL 9.5; AOLBuild 4337.35; Windows NT 5.1; .NET CLR 1.1.4322; .NET CLR 2.0.50727)",
            "Mozilla/5.0 (Windows; U; MSIE 9.0; Windows NT 9.0; en-US)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; Win64; x64; Trident/5.0; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 2.0.50727; Media Center PC 6.0)",
            "Mozilla/5.0 (compatible; MSIE 8.0; Windows NT 6.0; Trident/4.0; WOW64; Trident/4.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; .NET CLR 1.0.3705; .NET CLR 1.1.4322)",
            "Mozilla/4.0 (compatible; MSIE 7.0b; Windows NT 5.2; .NET CLR 1.1.4322; .NET CLR 2.0.50727; InfoPath.2; .NET CLR 3.0.04506.30)",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN) AppleWebKit/523.15 (KHTML, like Gecko, Safari/419.3) Arora/0.3 (Change: 287 c9dfb30)",
            "Mozilla/5.0 (X11; U; Linux; en-US) AppleWebKit/527+ (KHTML, like Gecko, Safari/419.3) Arora/0.6",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; en-US; rv:1.8.1.2pre) Gecko/20070215 K-Ninja/2.1.1",
            "Mozilla/5.0 (Windows; U; Windows NT 5.1; zh-CN; rv:1.9) Gecko/20080705 Firefox/3.0 Kapiko/3.0",
            "Mozilla/5.0 (X11; Linux i686; U;) Gecko/20070322 Kazehakase/0.4.5",
            "Mozilla/5.0 (X11; U; Linux i686; en-US; rv:1.9.0.8) Gecko Fedora/1.9.0.8-1.fc10 Kazehakase/0.5.6",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.56 Safari/535.11",
            "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_7_3) AppleWebKit/535.20 (KHTML, like Gecko) Chrome/19.0.1036.7 Safari/535.20",
            "Opera/9.80 (Macintosh; Intel Mac OS X 10.6.8; U; fr) Presto/2.9.168 Version/11.52",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/536.11 (KHTML, like Gecko) Chrome/20.0.1132.11 TaoBrowser/2.0 Safari/536.11",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.71 Safari/537.1 LBBROWSER",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E; LBBROWSER)",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/535.11 (KHTML, like Gecko) Chrome/17.0.963.84 Safari/535.11 LBBROWSER",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E; QQBrowser/7.0.3698.400)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 5.1; Trident/4.0; SV1; QQDownload 732; .NET4.0C; .NET4.0E; 360SE)",
            "Mozilla/4.0 (compatible; MSIE 6.0; Windows NT 5.1; SV1; QQDownload 732; .NET4.0C; .NET4.0E)",
            "Mozilla/4.0 (compatible; MSIE 7.0; Windows NT 6.1; WOW64; Trident/5.0; SLCC2; .NET CLR 2.0.50727; .NET CLR 3.5.30729; .NET CLR 3.0.30729; Media Center PC 6.0; .NET4.0C; .NET4.0E)",
            "Mozilla/5.0 (Windows NT 5.1) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.1 (KHTML, like Gecko) Chrome/21.0.1180.89 Safari/537.1",
            "Mozilla/5.0 (iPad; U; CPU OS 4_2_1 like Mac OS X; zh-cn) AppleWebKit/533.17.9 (KHTML, like Gecko) Version/5.0.2 Mobile/8C148 Safari/6533.18.5",
            "Mozilla/5.0 (Windows NT 6.1; Win64; x64; rv:2.0b13pre) Gecko/20110307 Firefox/4.0b13pre",
            "Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:16.0) Gecko/20100101 Firefox/16.0",
            "Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11",
            "Mozilla/5.0 (X11; U; Linux x86_64; zh-CN; rv:1.9.2.10) Gecko/20100922 Ubuntu/10.10 (maverick) Firefox/3.6.10",
            "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.36",
        ]


In [3]:
def contentUrl_to_displayUrl(contentUrl):
    if 'commons' in contentUrl:
        prefix = 'https://commons.wikimedia.org/wiki/'
    else:
        prefix = 'https://en.wikipedia.org/wiki/'
    tokens = contentUrl.split('/')
    if 'px-' in tokens[-1]:
        return prefix + 'File:'+tokens[-2]
    else:
        return prefix + 'File:'+tokens[-1]

In [4]:
def displayUrl_to_imgUrl(html):

    soup = BeautifulSoup(html, 'html.parser')
    result = soup.find_all('a',class_= "mw-thumbnail-link", limit=10)
    idx = 0
    default = ""
    for r in result:
        soup_short = BeautifulSoup(str(r), 'html.parser')
        for a in soup_short.find_all('a', href=True):
            href = a['href']
            if idx == 0: 
                default = a['href']
                idx += 1
            if '800px' in href:
                return href, "GOOD"
    if len(default)>0: return default, "GOOD"
    else:
        result = soup.find_all('div',class_= "fullImageLink", limit=1)
        for r in result:
            soup_short = BeautifulSoup(str(r), 'html.parser')
            for im in soup_short.find_all('img'):
                if max(int(im['width']), int(im['height']))>=600 and min(int(im['width']), int(im['height']))>=400:
                    return im['src'], "GOOD"
                return im['src'], "SIZE"
    return "", "FIELD_NONEXIST"

In [5]:
def get_display_page_html(desurl):
    start = time.time()
    try:
        req = urllib.request.Request(desurl, headers = {'User-Agent': random.choice(USER_AGENT_LIST)})
        with urllib.request.urlopen(req) as f:
            html = f.read().decode('utf-8')
        #print("url request time: ", time.time() - start)
        return html
    except KeyboardInterrupt:
        raise
    except:
        return ""

In [8]:
def normalize_imgUrl(url):
    if not 'commons' in url: return "https:"+url
    displayUrl = contentUrl_to_displayUrl(url)
    html = get_display_page_html(displayUrl)
    imgUrl, status = displayUrl_to_imgUrl(html)
    return imgUrl

In [17]:
data = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data/upd_txt_data_{}.json".format(1000), "r"))
try: data_upd_imgUrl = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "r"))
except: data_upd_imgUrl = {}
for k in range(50):
    if str(k) in data_upd_imgUrl: continue
    data_upd_imgUrl[str(k)] = copy.deepcopy(data[str(k)])
    data_upd_imgUrl[str(k)]['img_negFacts'] = []
    for f in data[str(k)]['img_negFacts']:
        if "maps.wikimedia.org" in f['imgUrl']: continue
        f_upd = copy.deepcopy(f)
        f_upd['imgUrl'] = normalize_imgUrl(f['imgUrl'])
        data_upd_imgUrl[str(k)]['img_negFacts'].append(f_upd)
    if k % 10 == 0:
        json.dump(data_upd_imgUrl, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "w"), indent=4)
json.dump(data_upd_imgUrl, open("/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/upd_txt_data_{}.json".format(1000), "w"), indent=4)

21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49


In [28]:
path = "/home/yingshac/CYS/WebQnA/WebQnA_data_new/upd_txt_data_upd_imgUrl/"

snippets_count = []
img_count = []

no_snippet_k = []
no_img_k = []

files = os.listdir(path)
data = {}
for f in files:
    if not '.json' in f: continue
    x = json.load(open(os.path.join(path, f), "r"))
    for k in x:
        num_snippets = len(x[k]['new_negFacts'])
        num_imgs = len(x[k]['img_negFacts'])
        if num_snippets == 0:
            no_snippet_k.append(k)
        if num_imgs == 0:
            no_img_k.append(k)
        snippets_count.append(num_snippets)
        img_count.append(num_imgs)
    data.update(x)
print("{} samples finished".format(len(data)))
print(sorted(Counter(snippets_count).items()))
print(sorted(Counter(img_count).items()))
print(len([k for k in data if len(data[k]['new_negFacts'])< 5 or len(data[k]['img_negFacts']) < 5]))

7921 samples finished
[(0, 51), (1, 24), (2, 35), (3, 27), (4, 28), (5, 38), (6, 66), (7, 46), (8, 56), (9, 49), (10, 72), (11, 62), (12, 50), (13, 49), (14, 57), (15, 71), (16, 52), (17, 56), (18, 58), (19, 54), (20, 59), (21, 47), (22, 65), (23, 46), (24, 61), (25, 46), (26, 58), (27, 50), (28, 52), (29, 40), (30, 50), (31, 52), (32, 59), (33, 40), (34, 54), (35, 61), (36, 47), (37, 59), (38, 44), (39, 59), (40, 5871)]
[(0, 36), (1, 85), (2, 85), (3, 99), (4, 119), (5, 431), (6, 415), (7, 380), (8, 350), (9, 319), (10, 307), (11, 283), (12, 239), (13, 255), (14, 221), (15, 213), (16, 225), (17, 160), (18, 178), (19, 177), (20, 142), (21, 135), (22, 136), (23, 125), (24, 123), (25, 103), (26, 103), (27, 92), (28, 84), (29, 83), (30, 80), (31, 74), (32, 61), (33, 63), (34, 74), (35, 82), (36, 76), (37, 82), (38, 102), (39, 224), (40, 1300)]
524


In [11]:
img_meta = json.load(open("/home/yingshac/CYS/WebQnA/WebQnA_data/img_metadata-Copy1.json", "r"))

In [21]:
for k in random.sample(list(img_meta.keys()), 20):
    if '.jpg' not in img_meta[k]['name'].lower():
        pprint(img_meta[k])

{'description': ' Canyon Road in Santa Fe - New Mexico - USA',
 'name': 'Canyon_Road_in_Santa_Fe,_New_Mexico,_USA',
 'page': 'https://commons.wikimedia.org/wiki/File:Canyon_Road_in_Santa_Fe%2C_New_Mexico%2C_USA.jpg',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/3/38/Canyon_Road_in_Santa_Fe%2C_New_Mexico%2C_USA.jpg/800px-Canyon_Road_in_Santa_Fe%2C_New_Mexico%2C_USA.jpg'}
{'description': '',
 'name': 'Peoples_Drug_Store,_interior,_11th_&_G,_(Washington,_D.C.)_LCCN2016823726.tif',
 'page': 'https://commons.wikimedia.org/wiki/File:Peoples_Drug_Store%2C_interior%2C_11th_%26_G%2C_%28Washington%2C_D.C.%29_LCCN2016823726.tif',
 'src': 'https://upload.wikimedia.org/wikipedia/commons/thumb/8/8a/Peoples_Drug_Store%2C_interior%2C_11th_%26_G%2C_%28Washington%2C_D.C.%29_LCCN2016823726.tif/lossy-page1-1200px-Peoples_Drug_Store%2C_interior%2C_11th_%26_G%2C_%28Washington%2C_D.C.%29_LCCN2016823726.tif.jpg'}
{'description': '',
 'name': "The_Missouri_float_in_President_Truman's_inaugural

In [24]:
print(max([int(k) for k in img_meta if int(k) < 10000000]))
print(min([int(k) for k in img_meta]))

22264
0
